# 팔머 펭귄
> x를 주면 펭귄을 분류하는 문제

* Iter1.
   - SVC 같은 선형 모델 선정
   - GridSearchCV
   - Pipeline

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("data/penguins_size.csv")
# df.sample(15)

In [3]:
df.isnull().sum()

species               0
island                0
culmen_length_mm      2
culmen_depth_mm       2
flipper_length_mm     2
body_mass_g           2
sex                  10
dtype: int64

In [4]:
penguins = df.dropna()
penguins.reset_index(drop=True, inplace=True) # 2 다음에 4가 나오면 안 된다. > reset_index
# penguins[penguins['sex'] == '.']
penguins.drop([327], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_7332\2054282674.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins.drop([327], inplace=True)


In [5]:
penguins['sex'].unique()

array(['MALE', 'FEMALE'], dtype=object)

In [6]:
penguins['species'] = penguins['species'].astype('category')
penguins['species_cod'] = penguins['species'].cat.codes

C:\Users\user\AppData\Local\Temp\ipykernel_7332\3177699378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species'] = penguins['species'].astype('category')
C:\Users\user\AppData\Local\Temp\ipykernel_7332\3177699378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species_cod'] = penguins['species'].cat.codes


In [7]:
penguins_feats = ['island', 'culmen_length_mm', 'culmen_depth_mm',
                  'flipper_length_mm', 'body_mass_g', 'sex']
penguins_labs = ['species_cod']
X, y = penguins[penguins_feats].values, penguins[penguins_labs].values

# SVC를 사용한 학습 및 평가

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

In [9]:
# 파이프라인 정의
n_feats = [1,2,3,4]
n_trans = Pipeline(steps=[('scaler', StandardScaler())])
c_feats = [0,5]
c_trans = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))]) #에러 떠도 무시해

In [10]:
# 파이프라인 실행
preprocessor = ColumnTransformer( #열 전체 변경
    transformers=[
        ('num', n_trans, n_feats), #파이프라인 실행해줄래, 대상 열
        ('cat', c_trans, c_feats)
    ]
)

In [11]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('svc', SVC(probability=True))])

In [12]:
SVC_scores = []
SVC_fimp = []
SVC_predictions = 0
FOLDS = 5

In [13]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [14]:
for fold, (train_idx, valid_idx) in enumerate(skf.split(penguins[penguins_feats].values,
                                                        penguins[penguins_labs].values)): #skf.split() : 인덱스가 있는 것들이 매개변수로
    print(f"Fold = {fold}")
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val_train, X_val, y_val_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    
    multi_model = pipeline.fit(X_val_train, y_val_train)
    preds_valid = multi_model.predict(X_val)
    acc = accuracy_score(y_val, preds_valid)
    SVC_scores.append(acc)

Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversio

In [15]:
np.mean(SVC_scores)

0.9850746268656716

In [16]:
confusion_matrix(y_val, preds_valid)

array([[32,  0,  0],
       [ 1, 13,  0],
       [ 0,  0, 21]], dtype=int64)